In [2]:
! pip install moviepy==1.0.3
! pip install ffmpeg==1.4


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import boto3
import uuid
import moviepy.editor as mp
import json

In [4]:
x = 'audio'
s3bucket= 'transcribe-output-x'

### Convert to WAV

In [5]:
def convert_mov(f):
    """Convert to file type Transcribe supports, also convert to wav to save space"""
    if f[-3:] not in ['wav','mp3']:
        print('Converting')
        my_clip = mp.VideoFileClip(f'{x}/{f}')
        print('Grabbed the clip')
        my_clip.audio.write_audiofile(f"{x}/{f[:-3]}wav")
        os.remove(f'{x}/{f}')
                                  

In [6]:
for f in os.listdir(x):
    if f != '.DS_Store' and '.ipynb' not in f:
        print(f)
        convert_mov(f)

MAF_20230921_CS2 RS25 engines installed~orig.wav


### Load to S3 and Transcribe

In [7]:
output_keys = []

for f in os.listdir(x):
    print(f)
    if f != '.DS_Store' and '.ipynb' not in f:
        s3_resource = boto3.client('s3')
        
        # LOAD VIDEOS TO S3
        s3object = f'{x}/{f}'
        s3_resource.upload_file(s3object, s3bucket, s3object)
        s3Path = "s3://" + s3bucket + "/" + s3object
        
        # PASS VIDEOS FROM S3 TO TANSCRIBE
        name = ''.join(s3object.split('.')[:-1]).split('/')[1].replace(" ", "")
        clean_name = ''.join(filter(str.isalnum, name))
        
        job_name = clean_name + '-' + str(uuid.uuid4())
        client = boto3.client('transcribe')
        
        outputkey = f'{x}/transcribe-output/' + clean_name + '.txt'
        print(f'Job output key is {outputkey}')
        output_keys.append(outputkey)
        
        response = client.start_transcription_job(
        TranscriptionJobName=job_name,
        LanguageCode='en-US',
        MediaFormat='wav',
        Media={
            'MediaFileUri': s3Path
        },
        OutputBucketName = s3bucket,
        OutputKey = outputkey,
    )

.ipynb_checkpoints
MAF_20230921_CS2 RS25 engines installed~orig.wav
Job output key is audio/transcribe-output/MAF20230921CS2RS25enginesinstalledorig.txt


### Process Output and Download Locally

In [8]:
output_keys

['audio/transcribe-output/MAF20230921CS2RS25enginesinstalledorig.txt']

In [9]:
write_to = 'source_documents'

for z in output_keys:
    print(f'Downloading {z} from S3')
    s3_response = s3_resource.get_object(Bucket = s3bucket, Key = z)
    s3_object_body = s3_response.get('Body')
    content = s3_object_body.read()
    json_dict = json.loads(content)
    data = json_dict['results']['transcripts'][0]['transcript']
    out = f"{write_to}/{z.split('/')[-1]}"
    print(f"Outputing to {out}")
    print('\n')
    
    with open(out, "w") as text_file:
        text_file.write(data)
        text_file.close()
    

Outputing to source_documents/MAF20230921CS2RS25enginesinstalledorig.txt


